In [1]:
import os, urllib, pickle
import urllib.parse
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
def loadfile(name):
    directory = str(os.getcwd())
    filepath = os.path.join(directory, name)
    with open(filepath, 'r', encoding = 'utf-8') as f:
        data = f.readlines()
    data = list(set(data))
    result = []

    for i in data:
        i = str(urllib.parse.unquote(i))
        result.append(i)
    
    return result

In [3]:
badQueries = loadfile('../dataset/XSS-WAF/badqueries.txt')
validQueries = loadfile('../dataset/XSS-WAF/goodqueries.txt')

In [4]:
badQueries = list(set(badQueries))
validQueries = list(set(validQueries))
allQueries = badQueries + validQueries

In [5]:
yBad = [1 for i in range(0, len(badQueries))]
yGood = [0 for i in range(0, len(validQueries))]

y = yBad + yGood

queries = allQueries

In [6]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3))

X = vectorizer.fit_transform(queries)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
badCount = len(badQueries)
validCount = len(validQueries)


In [12]:
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [13]:
p = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', lgs)
])

In [14]:
pickle.dump(p, open('trained_waf_model', 'wb'))

In [15]:
predicted = lgs.predict(X_test)

In [16]:
fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

In [17]:
print("Bad samples: %d" % badCount)
print("Good samples: %d" % validCount)
print("Baseline Constant negative: %.6f" % (float(validCount) / (validCount + badCount)))
print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

Bad samples: 45089
Good samples: 1265974
Baseline Constant negative: 0.965609
------------
Accuracy: 0.999279
Precision: 0.982713
Recall: 0.996293
F1-Score: 0.989456
AUC: 0.999969
